In [4]:
# magic function para hacer que los graficos de matplotlib se renderizen en el notebook.
%matplotlib inline

import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv

plt.style.use('default') # haciendo los graficos un poco mas bonitos xD
plt.rcParams['figure.figsize'] = (15, 5)


In [5]:
filesProperati = os.listdir("datos/properati")

In [6]:
dataFrames = []
for file_name in filesProperati:
    try:
        newDataFrame = pd.read_csv("datos/properati/"+file_name)
        dataFrames.append(newDataFrame)
    except:
        #Ver causa de este error (ParserError)
        #Error con el archivo: properati-AR-2015-06-01-properties-sell.csv
        #Error con el archivo: properati-AR-2015-08-01-properties-sell.csv
        #Error con el archivo: properati-AR-2015-04-01-properties-sell.csv
        #Error con el archivo: properati-AR-2015-07-01-properties-sell.csv
        print "Error con el archivo: "+file_name

Error con el archivo: properati-AR-2015-06-01-properties-sell.csv
Error con el archivo: properati-AR-2015-08-01-properties-sell.csv
Error con el archivo: properati-AR-2015-04-01-properties-sell.csv
Error con el archivo: properati-AR-2015-07-01-properties-sell.csv


In [7]:
#Hay algunos archivos de properati que tienen menos informacion, ver que hacer con ellos
masCols = []
for df in dataFrames:
    if (len(df.columns) == 27):
        masCols = df.columns
menosCols = []
for df in dataFrames:
    if (len(df.columns) == 20):
        menosCols = df.columns
diferencias = list(set(masCols) - set(menosCols))
#No tienen la superficie total en m2, asi que para estos archivos, 
#si no tenemos info del precio en alguna columna price, descartamos las filas
print diferencias


['surface_total_in_m2', 'description', 'title', 'surface_covered_in_m2', 'price_per_m2', 'country_name', 'id', 'state_name']


In [8]:
#¿que tipos de propiedades tenemos?
tiposDePropiedades = []
for df in dataFrames:
    try:
        propiedades = df[["property_type"]]
        tiposDePropiedades.append(propiedades)
    except KeyError:
        #No se por que me esta abriendo archivos que no son de properati, por eso este except
        pass

In [9]:
tiposDePropiedades = pd.concat(tiposDePropiedades)

In [10]:
tiposDePropiedades["property_type"].value_counts()

apartment    1222108
house         718179
PH            121444
store          37229
Name: property_type, dtype: int64

In [39]:
#Algo que se podria analizar es ver que tipo de propiedad tiene mayor precio promedio en los
#ultimos años. 

#hago lo mismo que hice para property_type con todas las columnas para ver si hay que limpiar
#algo
columnasEnComun = list(set.intersection(set(masCols),set(menosCols)))
descartar = ["properati_url", "image_thumbnail","lat",
             "expenses","price","price_usd_per_m2","price_aprox_usd",
             "lon", "geonames_id", "place_with_parent_names", "lat-lon", "created_on",
            "price_aprox_local_currency", "property_type"]
colsVerificar = list(set(columnasEnComun).difference(set(descartar)))
colsVerificar


['currency', 'operation', 'place_name', 'rooms', 'floor']

In [36]:
a = []
for feature in colsVerificar:
    featuresDfs= []
    for df in dataFrames:
        try:
            featureAct = df[[feature]]
            featuresDfs.append(featureAct)
        except KeyError:
            #No se por que me esta abriendo archivos que no son de properati, por eso este except
            pass
    a.append(featuresDfs)

In [49]:
concatenados = []
for featureAVerificar in a:
    concatenados.append(pd.concat(featureAVerificar))
for x in range(0,len(colsVerificar)):
    print "\n**********NEXT FEATURE************\n"
    print concatenados[x][colsVerificar[x]].value_counts()


**********NEXT FEATURE************

USD    1464198
ARS     228610
BRL        162
PEN          2
UYU          1
Name: currency, dtype: int64

**********NEXT FEATURE************

sell    2098960
Name: operation, dtype: int64

**********NEXT FEATURE************

Mar del Plata                      181204
Capital Federal                     96959
Rosario                             68420
Córdoba                             63819
Pilar                               57593
Tigre                               55979
Belgrano                            49534
Palermo                             43659
Ramos Mejía                         41526
Nordelta                            40819
La Plata                            38420
Caballito                           30557
Santa Fe                            30452
Ituzaingó                           28462
Morón                               27130
Olivos                              26284
Barrio Norte                        24037
Villa Urquiza            

In [ ]:
#ver de descartar esto (propiedades con #pisos irreales?)